# Q1. Gram-Schmidt Algorithm & QR Decomposition

1. Write a code to generate a random matrix A of size $m × n$ with $m > n$ and calculate its Frobenius norm, $∥ · ∥F$. The entries of A must be of the form r.dddd (example 5.4316). The inputs are the positive integers m and n and the output should display the the dimensions and the calculated norm value.


> **Deliverable(s) : The code with the desired input and output (0.5)**

In [1]:
from typing import List, Tuple
import random, math
import copy

def generate_random_entry() -> float:
    """[Returns a random number of the form r.dddd]

    Returns:
        float: A random number
    """

    digits = ""
    for i in range(5):
        dig = random.randint(0, 9)
        if i == 4: dig = random.randint(1, 9)
        digits = digits + str(dig)
        if i == 0: digits += "."
    return float(digits)

def create_random_matrix(m:int, n:int) -> List:
    """[Takes in the number of rows and columns and generates a matrix of size m x n]

    Args:
        m (int): Number of rows of the matrix
        n (int): Number of columns of the matrix

    Returns:
        List: A matrix as a list of lists
    """
    
    if m <= n:
        raise AssertionError("Requested matrix is square or wide; It must be tall. Please give m > n")
    
    A = []
    for i in range(m):
        ith_row = []
        for j in range(n):
            ith_row.append(generate_random_entry())
        A.append(ith_row)
    
    return A

def frob_norm(A: List) -> float:
    """[Takes in a matrix A and computes the frobenius norm of that matrix]

    Args:
        A (List): A matrix represented as List of Lists

    Returns:
        float: The frobenius norm of the matrix
    """
    
    if len(A) == 0:
        raise AssertionError("Empty matrix passed. Pass a finite matrix")

    fro_norm = 0.0

    for row in A:
        for element in row:
            fro_norm += element * element
    
    fro_norm = math.sqrt(fro_norm)
    return fro_norm

In [2]:
def display_matrix(A):
    """
    Given a matrix as a list of lists, just prints them out neatly with row and column numbers
    """
    nr = len(A)
    nc = len(A[0])
    matrix_string = "        "
    for col in range(nc):
        matrix_string += f"C{col+1:<7}"
    matrix_string += "\nR1   "
    for idx, r in enumerate(A):
        for element in r:
            matrix_string += f"{element: .4f} "
        if idx < nr - 1:
            matrix_string += f"\nR{idx + 2:<3} "
    print(matrix_string)

In [3]:
def rreduce(A:List, threshold:float = 1e-6) -> Tuple:
    """[Given a matrix A as a list of lists, compute it's row reduced echelon form.
    Return that along with the rank of the matrix]

    Args:
        A (List): A m x n tall matrix
        threshold (float, optional): When comparing the pivot elements, what magnitude of a value counts as zero magnitufe. Defaults to 1e-6.

    Returns:
        Tuple: A tuple of row reduced echelon form and the decision i.e. whether
        the matrix has linearly independent columns or not
    """
    # Create a copy of the original matrix
    A = copy.deepcopy(A)
            
    # Find out number of rows and columns
    nr, nc = len(A), len(A[0])            
    
    # Get Echelon form using row reduction (Only iterate for as many times as columns since tall matrix can't have more pivot elements than number of columns)
    for i in range(nc - 1):
        
        pivot_element = A[i][i]
        # TODO: Handle pivot = zero
        
        # Make all the elements below the pivot to be zero
        # in an iterative manner
        for j in range(i + 1, nr):
                        
            div_factor = A[j][i] / pivot_element     
            
            transformed_row = []
            for idx, element in enumerate(A[j]):
                transformed_element = element - div_factor * A[i][idx]
                if idx == i: transformed_element = 0.0
                transformed_row.append(transformed_element)
            
            A[j] = transformed_row
    
    # Check the leading diagonal entries;
    # If a few of them are zeros or near zeros,
    # then given matrix is not a full rank matrix
    full_rank = True
    for i in range(nc):
        if abs(A[i][i]) <= threshold:
            full_rank = False
            break
                    
    return (A, full_rank)

In [4]:
# Create a random tall matrix
A = create_random_matrix(5,4)
display_matrix(A)

        C1      C2      C3      C4      
R1    3.7677  2.9638  0.4322  9.9589 
R2    8.6845  1.2914  8.8298  6.6494 
R3    0.0977  1.1176  9.8997  0.2875 
R4    6.1157  9.7631  9.1169  9.0598 
R5    8.2658  2.0536  7.3457  1.5751 


In [10]:
# Row reduce the matrix into it's REF and see if it's columns are linearly independent
m, dec = rreduce(A)
display_matrix(m), dec

        C1      C2      C3      
R1    7.1179  1.9241  3.8554 
R2    0.0000  8.2676  1.0355 
R3    0.0000  0.0000  0.0000 
R4    0.0000  0.0000 -0.0000 


(None, False)

In [6]:
# Purposely generate a matrix where rows are linearly dependent
# And check if the REF is accurate or not
r1 = [generate_random_entry() for i in range(3)]
r2 = [generate_random_entry() for i in range(3)]
r3 = [0.1*(x + y) for x, y  in zip(r1, r2)]
r4 = [(x - y)/2 for x, y in zip(r2, r3)]
A = [r1,r2,r3,r4]

In [7]:
display_matrix(A)

        C1      C2      C3      
R1    7.1179  1.9241  3.8554 
R2    3.9372  9.3319  3.1681 
R3    1.1055  1.1256  0.7024 
R4    1.4158  4.1031  1.2329 


In [9]:
m, dec = rreduce(A)
display_matrix(m), dec

        C1      C2      C3      
R1    7.1179  1.9241  3.8554 
R2    0.0000  8.2676  1.0355 
R3    0.0000  0.0000  0.0000 
R4    0.0000  0.0000 -0.0000 


(None, False)